### 📦 第 0 格：导入必要的库
导入了如 `torch`, `cv2`, `numpy`, `matplotlib`, `tqdm`, `torchvision.transforms` 等用于图像处理、模型构建、训练可视化所需的基础包。

In [ ]:
import os  # 导入库
import cv2  # 导入库
import torch  # 导入库
import random  # 导入库
import numpy as np  # 导入库
import matplotlib.pyplot as plt  # 导入库
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

### ⚙️ 第 1 格：超参数配置
定义模型名称、训练轮数、批大小、学习率、测试集比例等训练配置参数。

In [ ]:
# 超参数设置
config = {  # 设置模型训练的超参数
    "model_name": "resnet",  # 可选 "resnet" 或 "vgg"
    "num_epochs": 15,
    "batch_size": 32,
    "learning_rate": 0.001,
    "test_size": 0.2,
    "sample_ratio": 1.0
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_path = "Aerial_Landscapes/"

### 🧠 第 2 格：模型初始化函数
根据传入的模型名（resnet 或 vgg）初始化预训练模型并替换输出层以匹配数据集类别数。

In [ ]:
def initialize_model(model_name, num_classes, use_pretrained=True):
    input_size = 224
    if model_name == "resnet":
        model = models.resnet18(pretrained=use_pretrained)  # 初始化 ResNet 模型
        model.fc = nn.Linear(model.fc.in_features, num_classes)  # 使用ResNet18
    elif model_name == "vgg":
        model = models.vgg16(pretrained=use_pretrained)  # 初始化 VGG 模型
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    else:
        raise ValueError("不支持的模型名称，请选择 'resnet' 或 'vgg'")
    return model, input_size

### 🧱 第 3 格：图像预处理流程
定义训练和测试图像所使用的图像增强与归一化操作，包括随机裁剪、缩放、标准化等。

In [ ]:
def get_transforms(input_size=224):
    train_transform = transforms.Compose([  # 定义图像预处理管道
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    # === 替换后的 test_transform: 加入模糊+噪声增强 ===

    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((input_size, input_size)),
        transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),  # ✅ 添加高斯模糊
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x + 0.05 * torch.randn_like(x)),    # ✅ 添加高斯噪声
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

# 原始 test_transform 被覆盖
# test_transform = transforms.Compose([  # 定义图像预处理管道
        transforms.ToPILImage(),
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return train_transform, test_transform

In [ ]:

# ✅ 定义遮挡函数：将图像中心区域遮成黑色，模拟遮挡情况
import torchvision.transforms.functional as TF
from PIL import ImageDraw

class RandomOcclusion:
    def __init__(self, size=(60, 60)):
        self.size = size  # 遮挡块尺寸

    def __call__(self, img):
        # 假设 img 是 PIL Image
        w, h = img.size
        x0 = w // 2 - self.size[0] // 2
        y0 = h // 2 - self.size[1] // 2
        x1 = x0 + self.size[0]
        y1 = y0 + self.size[1]
        draw = ImageDraw.Draw(img)
        draw.rectangle([x0, y0, x1, y1], fill=(0, 0, 0))
        return img



### 🧪 鲁棒性实验 2：图像遮挡测试

我们在测试集 transform 中加入 `RandomOcclusion` 操作，即在图像中心区域添加黑色遮挡块，用于模拟现实中摄像头被遮挡或图像不完整的情况。
通过评估模型在遮挡条件下的表现，可以测试其鲁棒性。


In [ ]:

# ✅ 应用遮挡增强 transform（测试时使用）
occluded_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    RandomOcclusion(size=(60, 60)),                     # 加遮挡块
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


### 🗂️ 第 4 格：自定义 PyTorch 数据集类
用于将图像和标签组合为 Dataset 对象，以便后续使用 DataLoader 加载。

In [ ]:
class CustomDataset(Dataset):  # 自定义数据集类
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, label

### 🧪 第 5 格：数据加载和划分函数
从指定文件夹加载图像数据，并按设定比例划分为训练集和测试集，同时返回类别标签映射。

In [ ]:
def load_and_split_dataset(root_dir, test_size=0.2, sample_ratio=1.0):
    classes = sorted(os.listdir(root_dir))
    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
    train_images, train_labels = [], []
    test_images, test_labels = [], []

    for cls_name in classes:
        cls_path = os.path.join(root_dir, cls_name)
        img_files = [os.path.join(cls_path, f) for f in os.listdir(cls_path) if f.endswith('.jpg')]
        random.seed(42)
        random.shuffle(img_files)
        n_samples = int(len(img_files) * sample_ratio)
        split = int(n_samples * (1 - test_size))
        imgs = img_files[:n_samples]
        train_imgs = imgs[:split]
        test_imgs = imgs[split:]
        train_images.extend([cv2.imread(p) for p in train_imgs])
        train_labels.extend([class_to_idx[cls_name]] * len(train_imgs))
        test_images.extend([cv2.imread(p) for p in test_imgs])
        test_labels.extend([class_to_idx[cls_name]] * len(test_imgs))

    return (train_images, train_labels), (test_images, test_labels), classes

### 🔁 第 6 格：ResNet/VGG 训练函数
包括训练循环、Early Stopping、loss 和 acc 的记录，并在训练完成后绘制学习曲线图。

In [ ]:
from tqdm import tqdm

def train_model(model, device, train_loader, test_loader, criterion, optimizer,  # 定义 ResNet/VGG 通用训练函数
                num_epochs=25, checkpoint_path='checkpoint.pth', patience=5):

    best_acc = 0.0
    epochs_no_improve = 0
    early_stop = False
    history = {
        'train_loss': [], 'train_acc': [],
        'test_loss': [], 'test_acc': []
    }

    for epoch in range(num_epochs):
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        for batch in train_loader_tqdm:
            if early_stop:
                print(f"⚠️ 早停触发于第 {epoch+1} 轮")
                break

        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_train += torch.sum(preds == labels.data)
            total_train += labels.size(0)

        epoch_train_loss = train_loss / total_train
        epoch_train_acc = correct_train.double() / total_train
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc.item())

        model.eval()
        test_loss, correct_test, total_test = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                test_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct_test += torch.sum(preds == labels.data)
                total_test += labels.size(0)

        epoch_test_loss = test_loss / total_test
        epoch_test_acc = correct_test.double() / total_test
        history['test_loss'].append(epoch_test_loss)
        history['test_acc'].append(epoch_test_acc.item())

        print(f"Epoch {epoch+1}: Train Acc={epoch_train_acc:.4f}, Test Acc={epoch_test_acc:.4f}")

        if epoch_test_acc > best_acc:
            best_acc = epoch_test_acc
            epochs_no_improve = 0
            torch.save(model.state_dict(), checkpoint_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                early_stop = True

    return model, history
    # ✅ 添加训练与验证的损失/准确率曲线图像
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure(figsize=(10, 5))
    plt.plot(history['train_loss'], label='Train Loss', c='red')
    plt.plot(history['test_loss'], label='Val Loss', c='blue')
    plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.legend(); plt.title('Loss Curve')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(history['train_acc'], label='Train Acc', c='orangered')
    plt.plot(history['test_acc'], label='Val Acc', c='green')
    plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.legend(); plt.title('Accuracy Curve')
    plt.show()


### 📊 第 7 格：评估函数
对模型在测试集上的表现进行评估，生成预测值与真实值列表，可用于后续可视化。

In [ ]:
def evaluate_model(model, test_loader, device, class_names):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels)
            y_pred.extend(preds.cpu().numpy())

    print(classification_report(y_true, y_pred, target_names=class_names))
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(xticks_rotation='vertical', cmap='Blues')

### 🧪 第 8 格：加载数据集
调用数据加载函数并获取训练图像、测试图像、类别标签等信息。

In [ ]:
(train_images, train_labels), (test_images, test_labels), classes = load_and_split_dataset(
    dataset_path,
    test_size=config["test_size"],  # 设置模型训练的超参数
    sample_ratio=config["sample_ratio"]  # 设置模型训练的超参数
)

model, input_size = initialize_model(
    model_name=config["model_name"],  # 设置模型训练的超参数
    num_classes=len(classes),
    use_pretrained=True
)
model = model.to(device)

train_transform, test_transform = get_transforms(input_size)

train_dataset = CustomDataset(train_images, train_labels, train_transform)
test_dataset = CustomDataset(test_images, test_labels, test_transform)
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)  # 设置模型训练的超参数
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)  # 设置模型训练的超参数

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])  # 设置模型训练的超参数

model, history = train_model(
    model, device, train_loader, test_loader,
    criterion, optimizer,
    num_epochs=config["num_epochs"],  # 设置模型训练的超参数
    patience=5
)

model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels)
        y_pred.extend(preds.cpu().numpy())

report = classification_report(y_true, y_pred, target_names=classes, digits=4)
print("Evaluation Metrics (Precision, Recall, F1-score):\n")
print(report)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
plt.figure(figsize=(12, 10))
disp.plot(xticks_rotation=45, cmap='Blues')
plt.title("Confusion Matrix")
plt.grid(False)
plt.tight_layout()
plt.show()

### ▶️ 第 9 格：run_and_evaluate 函数
这是一个便捷函数，用于初始化模型、训练并评估结果，适用于快速比较多个模型表现。

In [ ]:
def run_and_evaluate(model_name, classes):  # 执行模型训练和评估
    (train_images, train_labels), (test_images, test_labels), _ = load_and_split_dataset(
        dataset_path,
        test_size=config["test_size"],  # 设置模型训练的超参数
        sample_ratio=config["sample_ratio"]  # 设置模型训练的超参数
    )

    model, input_size = initialize_model(
        model_name=model_name,
        num_classes=len(classes),
        use_pretrained=True
    )
    model = model.to(device)

    train_transform, test_transform = get_transforms(input_size)

    train_dataset = CustomDataset(train_images, train_labels, train_transform)
    test_dataset = CustomDataset(test_images, test_labels, test_transform)
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)  # 设置模型训练的超参数
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)  # 设置模型训练的超参数

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])  # 设置模型训练的超参数

    model, _ = train_model(
        model, device, train_loader, test_loader,
        criterion, optimizer,
        num_epochs=config["num_epochs"],  # 设置模型训练的超参数
        patience=5
    )

    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels)
            y_pred.extend(preds.cpu().numpy())

    report = classification_report(y_true, y_pred, target_names=classes, digits=4)
    print(f"===== {model_name.upper()} Evaluation Metrics =====\n")
    print(report)

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    plt.figure(figsize=(12, 10))
    disp.plot(xticks_rotation=45, cmap='Blues')
    plt.title(f"Confusion Matrix for {model_name.upper()}")
    plt.grid(False)
    plt.tight_layout()
    plt.show()

In [ ]:
print("🔍 正在比较 ResNet18 与 VGG16 的分类性能...\n")
run_and_evaluate("resnet", classes)  # 执行模型训练和评估

In [ ]:
def train_vgg_model(model, device, train_loader, val_loader, criterion, optimizer, num_epochs=10):  # 定义 VGG 的独立训练函数
    train_losses, val_losses, train_accs, val_accs = [], [], [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):  # 加 tqdm 显示训练进度
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        train_losses.append(running_loss / len(train_loader))
        train_accs.append(correct / total)

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        val_losses.append(val_loss / len(val_loader))
        val_accs.append(val_correct / val_total)

        print(f"[VGG] Epoch {epoch+1}/{num_epochs} - Train Acc: {train_accs[-1]:.4f} - Val Acc: {val_accs[-1]:.4f}")

    import matplotlib.pyplot as plt  # 导入库
    import numpy as np  # 导入库
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label="Train Loss", c="red")
    plt.plot(val_losses, label="Val Loss", c="blue")
    plt.xlabel("Epochs"); plt.ylabel("Loss"); plt.legend(); plt.show()
    plt.figure(figsize=(10, 5))
    plt.plot(train_accs, label="Train Acc", c="orangered")
    plt.plot(val_accs, label="Val Acc", c="green")
    plt.xlabel("Epochs"); plt.ylabel("Accuracy"); plt.legend(); plt.show()

In [ ]:
vgg_model = initialize_model("vgg", num_classes=len(classes))[0].to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg_model.parameters(), lr=config["learning_rate"])  # 设置模型训练的超参数

train_vgg_model(vgg_model, device, train_loader, test_loader, criterion, optimizer, num_epochs=config["num_epochs"]) # 设置模型训练的超参数

In [ ]:
# ✅ 使用 evaluate_model 函数评估 VGG 模型分类表现
print("\n==================== VGG Evaluation Metrics ====================")
evaluate_model(vgg_model, test_loader, device, class_names=classes)
plt.title(f"Confusion Matrix for VGG16")

In [ ]:
import os  # 导入库
import cv2  # 导入库
import torch  # 导入库
import numpy as np  # 导入库
import matplotlib.pyplot as plt  # 导入库
from collections import defaultdict
from torchcam.methods import GradCAM

def overlay_heatmap(img: np.ndarray, cam: np.ndarray, alpha: float = 0.5) -> np.ndarray:
    cam_uint8 = np.uint8(255 * cam)
    heatmap = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    if img.max() > 1.0:
        img = np.float32(img) / 255
    if img.shape[:2] != heatmap.shape[:2]:
        heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    overlayed = heatmap * alpha + img
    overlayed = overlayed / np.max(overlayed)
    return np.uint8(255 * overlayed)

def unnormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

def apply_gradcam_all_classes(model, device, dataloader, model_name="resnet", save_dir="gradcam_outputs"):
    os.makedirs(save_dir, exist_ok=True)
    model.eval()

    if model_name == "resnet":
        target_layer = model.layer4[-1].conv2
    elif model_name == "vgg":
        target_layer = model.features[-1]
    else:
        raise ValueError("Unsupported model")

    cam_extractor = GradCAM(model, target_layer=target_layer)

    seen_classes = defaultdict(int)
    total_target_classes = 15
    class_id_to_name = [
        "Agriculture", "Airport", "Beach", "City", "Desert", "Forest", "Grassland", "Highway",
        "Lake", "Mountain", "Parking", "Port", "Railway", "Residential", "River"
    ]

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        for i in range(inputs.shape[0]):
            label = labels[i].item()
            if seen_classes[label] >= 1:
                continue 

            img_tensor = inputs[i].unsqueeze(0)
            output = model(img_tensor)
            class_idx = torch.argmax(output).item()

            cam_tensor = cam_extractor(class_idx=class_idx, scores=output)[0]
            cam = cam_tensor.cpu().numpy()
            if cam.ndim == 3:
                cam = cam[0]
            cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

            unnorm_img_tensor = unnormalize(img_tensor.squeeze(0).cpu(),
                                            mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225])
            raw_image = np.clip(unnorm_img_tensor.permute(1, 2, 0).numpy(), 0, 1)

            cam = cv2.resize(cam, (raw_image.shape[1], raw_image.shape[0]))
            result = overlay_heatmap(raw_image, cam)
            result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

            orig_img = np.uint8(raw_image * 255)
            orig_bgr = cv2.cvtColor(orig_img, cv2.COLOR_RGB2BGR)
            result_bgr = cv2.cvtColor(result_rgb, cv2.COLOR_RGB2BGR)

            cname = class_id_to_name[label]
            cv2.imwrite(os.path.join(save_dir, f"{cname}_original.png"), orig_bgr)
            cv2.imwrite(os.path.join(save_dir, f"{cname}_gradcam.png"), result_bgr)
            cv2.imwrite(os.path.join(save_dir, f"{cname}_compare.png"), np.hstack((orig_bgr, result_bgr)))

            fig, axs = plt.subplots(1, 2, figsize=(10, 4))
            axs[0].imshow(orig_img)
            axs[0].set_title(f"{cname} - Original")
            axs[0].axis('off')
            axs[1].imshow(result_rgb)
            axs[1].set_title(f"{cname} - GradCAM (Pred: {class_id_to_name[class_idx]})")
            axs[1].axis('off')
            plt.tight_layout()
            plt.show()

            seen_classes[label] += 1

        if len(seen_classes) >= total_target_classes:
            print("✅ 已为所有类别生成 Grad-CAM 可视化。")
            break

In [ ]:
apply_gradcam_all_classes(model, device, test_loader, model_name="resnet")


## 🧪 模型鲁棒性对比评估：ResNet vs VGG

我们使用 `occluded_test_transform`（中心遮挡 + 归一化）对两个模型进行评估，比较它们在遮挡图像上的分类性能，包括：
- 准确率（Accuracy）
- 精确率 / 召回率 / F1 分数（Precision / Recall / F1-score）

通过这个实验可以观察不同架构的鲁棒性差异。


In [ ]:

# ✅ 构建遮挡数据加载器
occluded_dataset = CustomDataset(test_images, test_labels, transform=occluded_test_transform)
occluded_loader = DataLoader(occluded_dataset, batch_size=32, shuffle=False)

# ✅ 对 ResNet 模型在遮挡数据上评估
print("🔍 ResNet under Occlusion")
evaluate_model(resnet_model, occluded_loader, device, class_names=classes)

# ✅ 对 VGG 模型在遮挡数据上评估
print("\n🔍 VGG under Occlusion")
evaluate_model(vgg_model, occluded_loader, device, class_names=classes)



## 🔁 交叉验证（Cross-Validation）支持

为了增强模型性能评估的稳健性，我们可以使用 **5-fold 交叉验证（5-Fold CV）**。
- 将整个训练数据划分为 5 个子集（folds）
- 每次选择其中一个作为验证集，其他 4 个作为训练集
- 训练 5 次，计算平均准确率和 F1-score

这避免了单一训练/验证划分导致的偶然偏差，提升模型评估的可靠性。



## 📈 每折训练过程可视化（Loss & Accuracy）

为了观察模型在每一折训练/验证过程中的稳定性，我们记录了每一折的训练与验证 Loss/Accuracy 曲线。
这有助于判断：
- 模型是否存在过拟合或欠拟合
- 不同折之间训练过程是否稳定一致
- 模型在少量数据上的泛化能力是否良好


In [ ]:

acc_list, f1_list = [], []

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"📂 Fold {fold+1}/5")
    
    X_train, y_train = image_paths[train_idx], labels[train_idx]
    X_val, y_val = image_paths[val_idx], labels[val_idx]

    train_ds = CustomDataset(X_train, y_train, transform=train_transform)
    val_ds = CustomDataset(X_val, y_val, transform=test_transform)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

    model = initialize_model("vgg", num_classes=len(classes))[0].to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

    # 每折都记录曲线
    history = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}

    for epoch in range(5):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        history['train_loss'].append(running_loss / len(train_loader))
        history['train_acc'].append(correct / total)

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        history['test_loss'].append(val_loss / len(val_loader))
        history['test_acc'].append(val_correct / val_total)

    # 绘制每折训练/验证曲线
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure(figsize=(10, 4))
    plt.plot(history['train_loss'], label='Train Loss', c='red')
    plt.plot(history['test_loss'], label='Val Loss', c='blue')
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.title(f'Fold {fold+1} - Loss Curve')
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.plot(history['train_acc'], label='Train Acc', c='orangered')
    plt.plot(history['test_acc'], label='Val Acc', c='green')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy')
    plt.title(f'Fold {fold+1} - Accuracy Curve')
    plt.legend()
    plt.show()

    # 记录评估指标
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    acc_list.append(acc)
    f1_list.append(f1)
    print(f"Fold {fold+1} Accuracy: {acc:.4f}, F1-score: {f1:.4f}\n")

print("✅ 平均准确率:", np.mean(acc_list))
print("✅ 平均加权F1分数:", np.mean(f1_list))
